In [14]:
# 필요한 패키지 설치
!pip install -U langchain-community
!pip install openai langchain pypdf tiktoken sentence-transformers chromadb

In [ ]:
# 필요한 라이브러리 임포트
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
import numpy as np

In [16]:
# 코사인 유사도 함수 정의
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [17]:
# 토크나이저 설정
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [18]:
# PDF 파일 로드 및 청크로 분할
pdf_path = "path_to_your_pdf_file.pdf"  # 여기에 PDF 파일 경로를 넣기
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

ValueError: File path path_to_your_pdf_file.pdf is not a valid file or url

In [ ]:
# 텍스트 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

## 한국어 사전 학습 모델임베딩 -ko-sbert-nli

In [ ]:
# 오픈 소스 임베딩 함수 생성
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': 'True'}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

## Chroma

In [ ]:
# 문서 텍스트 추출 및 임베딩 생성
doc_texts = [doc.page_content for doc in docs]
doc_embeddings = hf.embed_documents(doc_texts)

# Chroma 데이터베이스에 임베딩 저장
db = Chroma.from_documents(docs, hf, persist_directory="./chroma_db")

In [ ]:
# 쿼리 생성 및 임베딩
query = "질문"
query_embedding = hf.embed_query(query)

In [ ]:
# Chroma 데이터베이스 불러오기
db3 = Chroma(persist_directory="./chroma_db", embeddings_function=hf)

# 유사도 계산 및 유사한 문서 검색
docs_with_scores = db3.similarity_search_with_relevance_score(query, k=3)


In [ ]:
# 결과 출력
print("가장 유사한 문서: \n\n{}\n\n".format(docs_with_scores[0][0].page_content))
print("문서 유사도: \n {:.2f}".format(docs_with_scores[0][1]))